In [1]:
FLAG_GEOCODE = False

## Data acquisition

[FEMA Nnational Flood hazard Layer (NFHL) data download](https://msc.fema.gov/portal/advanceSearch#searchresultsanchor)

[FEMA Nnational Flood hazard Layer (NFHL) interactive viewer](https://hazards-fema.maps.arcgis.com/apps/webappviewer/index.html?id=8b0adb51996444d4879338b5529aa9cd)

In [2]:
import utils
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

Load FEMA Flood Zone Shapefile
Look for something like 'X500' or '0.2 PCT ANNUAL CHANCE FLOOD HAZARD'.

In [3]:
# Load FEMA flood zones
flood_zones = {}
flood_zones['ny_albany'] = gpd.read_file("data//36001C_20190306_ny_albany//S_FLD_HAZ_AR.shp")[['FLD_ZONE', 'ZONE_SUBTY', 'geometry']]
flood_zones['ct_fairfield'] = gpd.read_file("data//09001C_20250115_ct_fairfield//S_FLD_HAZ_AR.shp")[['FLD_ZONE', 'ZONE_SUBTY', 'geometry']]

## EDA

In [5]:
print(f'unique FLD_ZONE: {flood_zones["ny_albany"]["FLD_ZONE"].unique()}')
print(f'unique ZONE_SUBTY: {flood_zones["ny_albany"]["ZONE_SUBTY"].unique()}')

# compare ZONE_SUBTY and FLD_ZONE:
df = flood_zones['ny_albany'].copy()
df["FLD_ZONE == ZONE_SUBTY"] = (df["FLD_ZONE"].str.contains("X", case=False).values == df["ZONE_SUBTY"].str.contains("0.2 pct", case=False).values)
df[["FLD_ZONE", "ZONE_SUBTY", "FLD_ZONE == ZONE_SUBTY"]].head()

unique FLD_ZONE: ['X' 'AE' 'A' 'AO']
unique ZONE_SUBTY: ['0.2 PCT ANNUAL CHANCE FLOOD HAZARD' None 'AREA OF MINIMAL FLOOD HAZARD'
 'FLOODWAY']


,FLD_ZONE,ZONE_SUBTY,FLD_ZONE == ZONE_SUBTY
0,X,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,True
1,AE,None,False
2,X,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,True
3,AE,None,False
4,AE,None,False


| Column       | Purpose                                                            |
| ------------ | ------------------------------------------------------------------ |
| `FLD_ZONE`   | Main flood zone classification (e.g. X500, AE)                     |
| `ZONE_SUBTY` | Subtype, e.g. "0.2 PCT ANNUAL CHANCE..."                           |
| `SFHA_TF`    | Special Flood Hazard Area: `T` if in high-risk 100-year floodplain |
| `STATIC_BFE` | Base Flood Elevation (if available)                                |
| `geometry`   | Polygon geometry for spatial join                                  |

__Zones show up as:__
- ZONE_SUBTY = "0.2 pct annual chance flood hazard"
- FLD_ZONE = "X",
- or sometimes FLD_ZONE = "X500" in older maps

## Filter the 100- and 500-Year Flood Zones

In [6]:
for state_county in flood_zones.keys():
    df = flood_zones[state_county]
    df['rp100'] = df['FLD_ZONE'].isin(['A', 'AE', 'AO'])

    df['rp500'] = (
    # df['FLD_ZONE'].str.contains('X', case=False, na=False) |
    df['ZONE_SUBTY'].str.contains('0.2 pct', case=False, na=False)
    )

    df.drop(columns=['FLD_ZONE', 'ZONE_SUBTY'], inplace=True)

flood_zones["ny_albany"].head(2)

,geometry,rp100,rp500
0,"POLYGON ((-74.07049 42.48556, -74.07049 42.485...",False,True
1,"POLYGON ((-74.03238 42.50037, -74.03237 42.500...",True,False


In [8]:
assets = {}
assets['ny_albany'] = pd.read_csv("data/assets_ny_albany.csv")
assets['ct_fairfield'] = pd.read_csv("data/assets_ct_fairfield.csv")
assets['ct_fairfield'].head(1)

,id,address,description,lat_true,lon_true
0,1,"1 Reef Rd, Fairfield, CT 06824",(outside),41.140852,-73.25729


## Geocode the Addresses to Lat/Lon Points

In [10]:
# Geocode each address with retry logic (📌if have lat/lon coordinates - skip geocoding)
if FLAG_GEOCODE:
    for state_county in assets.keys():
        utils.geocode_assets(assets[state_county], max_retries=3, timeout=3, sleep_time=2)
        assets[state_county].to_csv(f"data/assets_{state_county}_geocoded.csv", index=False)
else:
    for state_county in assets.keys():
        assets[state_county] = pd.read_csv(f"data/assets_{state_county}_geocoded.csv")
assets['ct_fairfield'].head(1)

,id,address,description,lat_true,lon_true,lat,lon
0,1,"1 Reef Rd, Fairfield, CT 06824",(outside),41.140852,-73.25729,41.140852,-73.25729


In [11]:
# add geometry column
for state_county in assets.keys():
    assets[state_county] = utils.add_geometry_column(assets[state_county])
    assets[state_county].drop(columns=['lat_true', 'lon_true', 'lat', 'lon'], inplace=True)
assets['ct_fairfield'].head(1)

,id,address,description,geometry
0,1,"1 Reef Rd, Fairfield, CT 06824",(outside),POINT (-73.25729 41.14085)


Ensure Same CRS and Perform Spatial Join

In [12]:
# Reproject FEMA flood zones to match address CRS if needed
for state_county in assets.keys():
    flood_zones[state_county] = flood_zones[state_county].to_crs(assets[state_county].crs)

## Spatial Join of Flood_Zones & Asset_Locations

In [13]:
# Spatial join to find assets in RP100 flood zones: for them index_right is not null
# Add boolean column: True if matched a polygon, else False
for state_county in assets.keys():
    assets[state_county]['rp100'] = gpd.sjoin(assets[state_county], 
                        flood_zones[state_county][flood_zones[state_county]['rp100']],
                        how='left', predicate='intersects').index_right.notna()
    assets[state_county]['rp500'] = gpd.sjoin(assets[state_county], 
                        flood_zones['ct_fairfield'][flood_zones['ct_fairfield']['rp500']],
                        how='left', predicate='intersects').index_right.notna()
    conditions = [assets[state_county]['rp100']==True, assets[state_county]['rp500']==True]
    choices = ['rp100', 'rp500']
    assets[state_county]['flood_zone'] = np.select(conditions, choices, default='outside')
assets['ct_fairfield'].head(1)

,id,address,description,geometry,rp100,rp500,flood_zone
0,1,"1 Reef Rd, Fairfield, CT 06824",(outside),POINT (-73.25729 41.14085),False,False,outside


In [14]:
assets['ny_albany']

,id,address,description,geometry,rp100,rp500,flood_zone
0,1,"1 Quay St, Albany, NY 12207",rp100,POINT (-73.74477 42.6537),True,False,rp100
1,2,"140 State St, Albany, NY 12207",outside,POINT (-73.7553 42.65093),False,False,outside
2,3,"44 Holland Ave, Albany, NY 12229",outside,POINT (-73.77375 42.64808),False,False,outside
3,4,"183 Schoolhouse Rd, Albany, NY 12203",outside,POINT (-73.84945 42.67324),False,False,outside
4,5,"42 S Dove St, Albany, NY 12202",outside,POINT (-73.77033 42.64167),False,False,outside
5,6,"11 N Pearl St, Albany, NY 12207",outside,POINT (-73.75194 42.65018),False,False,outside
6,7,"600 Broadway, Albany, NY 12207",rp500,POINT (-73.74961 42.65229),False,False,outside
7,8,"126 State St, Albany, NY 12207",outside,POINT (-73.75452 42.65068),False,False,outside
8,9,"186 Fuller Rd, Albany, NY 12205",outside,POINT (-73.82853 42.696),False,False,outside
9,10,"106 Smith Blvd, Albany, NY 12202",rp100,POINT (-73.75803 42.62648),True,False,rp100


In [15]:
assets['ct_fairfield']

,id,address,description,geometry,rp100,rp500,flood_zone
0,1,"1 Reef Rd, Fairfield, CT 06824",(outside),POINT (-73.25729 41.14085),False,False,outside
1,3,"385 Fairfield Beach Rd, Fairfield, CT 06824",(rp100),POINT (-73.24191 41.1345),True,False,rp100
2,10,"200 Mill Hill Rd, Southport, CT 06890",(outside),POINT (-73.28188 41.14281),False,False,outside
3,2,"100 Pequot Ave, Southport, CT 06890",(outside),POINT (-73.28131 41.13894),False,False,outside
4,8,"834 Brookside Dr, Fairfield, CT 06824",(rp500),POINT (-73.26761 41.17737),False,False,outside
5,5,"200 Reef Rd, Fairfield, CT 06824",(rp100),POINT (-73.25617 41.13859),True,False,rp100
6,4,"6 Alma Dr, Fairfield, CT, 06824, USA",(rp500),POINT (-73.27168 41.16653),False,True,rp500
7,6,"308 Alma Dr, Fairfield, CT, 06824, USA",(rp100),POINT (-73.27027 41.16672),True,False,rp100
8,7,"1073 N Benson Rd, Fairfield, CT 06824",(outside),POINT (-73.25282 41.15964),False,False,outside
9,9,"905 Kings Hwy E, Fairfield, CT 06825",(rp500),POINT (-73.23335 41.16724),False,False,outside


## Plot results for  Fairfield, CT

In [16]:
flood_gdf = flood_zones['ct_fairfield']
asset_gdf = assets['ct_fairfield']

In [ ]:
# create an interactive map of flood risk using folium (save to HTML file)
utils.plot_flood_risk(assets['ct_fairfield'], flood_zones['ct_fairfield'], save_to_filename="results/flood_zones_map.html")

In [24]:
# Export to Shapefile (.shp)
asset_gdf.to_file("results/esri_shapefile/assets_with_flood_info.shp", driver="ESRI Shapefile")
# Export to GeoPackage (.gpkg)
asset_gdf.to_file("results/assets_with_flood_info.gpkg", layer="assets", driver="GPKG")

C:\Users\andri\AppData\Local\Temp\ipykernel_40228\2733062726.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  asset_gdf.to_file("results/esri_shapefile/assets_with_flood_info.shp", driver="ESRI Shapefile")
d:\_DT\Prog\_MyCode\GIS\.venv\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'description' to 'descriptio'
  ogr_write(


# <span style="color:red"> __WIP__ </span>

<span style="color:red"> Optional Improvements </span>
| Task                                            | Tool                                       |
| ----------------------------------------------- | ------------------------------------------ |
| Save result shapefile                 | `.to_file(...)` |
| More robust spatial join (buffering, proximity) | Use `.buffer()` or distance queries        |
|Is there an API to query if a loc's flood zones?| `TBD`|

In [ ]:
# import matplotlib.pyplot as plt

# # Spatial join: find assets in flood zone
# assets_in_zone = gpd.sjoin(gdf_assets, flood_500, how="inner", predicate="intersects")
# assets_out_zone = gdf_assets[~gdf_assets.index.isin(assets_in_zone.index)]

# # Plot
# fig, ax = plt.subplots(figsize=(10, 10))
# flood_500.plot(ax=ax, color='lightblue', alpha=0.5, edgecolor='blue', linewidth=0.5, label='500-yr Flood Zone')
# assets_out_zone.plot(ax=ax, color='green', markersize=40, label='Assets Outside Flood Zone')
# assets_in_zone.plot(ax=ax, color='red', markersize=40, label='Assets Inside Flood Zone')

# plt.legend()
# plt.title("Asset Exposure to 500-Year Flood Zone")
# plt.axis('off')
# plt.show()